In [1]:
from strategy.speculation import Speculation
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p

In [2]:
start_date = datetime(2015,1,1)
end_date = datetime(2021,1,1)
spec = Speculation(start_date,end_date)

In [3]:
spec.subscribe()

In [4]:
spec.subscriptions

{'market': {'db': <database.market.Market at 0x256ffca38b0>},
 'stock_category': {'db': <database.product.Product at 0x256ffca36d0>}}

In [5]:
spec.db

In [6]:
transformed = spec.initial_transform()

In [7]:
sim = spec.create_sim()

In [9]:
sim

,date,ticker,adjclose,delta,score,timeframe,score_requirement,requirement,category_training_year,model_training_year,number_of_training_weeks,value
0,2015-01-02,AAPL,24.779987,0.011755,0.994496,weekly,70,5,4,1,14,True
1,2015-01-05,AAPL,24.081896,0.035121,0.994496,weekly,70,5,4,1,14,True
2,2015-01-06,AAPL,24.084162,0.035023,0.994496,weekly,70,5,4,1,14,True
3,2015-01-07,AAPL,24.421875,0.020711,0.994496,weekly,70,5,4,1,14,True
4,2015-01-08,AAPL,25.360219,-0.017056,0.994496,weekly,70,5,4,1,14,True
...,...,...,...,...,...,...,...,...,...,...,...,...
77257,2020-12-24,MA,334.789604,-0.021497,0.968322,weekly,70,5,4,1,14,True
77258,2020-12-28,MA,343.229092,0.023767,0.968322,weekly,70,5,4,1,14,True
77259,2020-12-29,MA,345.411203,0.017300,0.968322,weekly,70,5,4,1,14,True
77260,2020-12-30,MA,354.269178,-0.008136,0.968322,weekly,70,5,4,1,14,True


In [8]:
sim["delta"].mean()

4.1706288615140254e+30